### Credits
This notebook was created by Laura Arditti and Leonardo Cianfanelli.

## Spanning subgraphs
A spanning subgraph is a subgraph that contains all the nodes of the original graph. Of particular interest are spanning trees, i.e., spanning subgraphs that are trees. Spanning trees find many applications, e.g., traveling salesman problem, and search algorithms.

We can generate all the spanning subgraphs of a given graph as follows.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

# As an example, construct a graph
G = nx.Graph()
nx.add_cycle(G, range(4))
G.add_edges_from([(0,2), (1,3)]) 
nx.draw_circular(G, with_labels=True)

# Define a function for computing the power set of a given set
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

# Use 'powerset' to compute all subgraphs of G
subgraphs = list()
for edge_set in powerset(G.edges):
    H = nx.Graph()
    H.add_nodes_from(G.nodes)
    H.add_edges_from(edge_set)
    subgraphs.append(H)
print("Number of subgraphs:", len(subgraphs))

We now compute all spanning subtrees of G. To this end, exploit the following characterization:

**Theorem**: an undirected graph T is a tree if and only if it is connected and E=N-1, where E is the number of (undirected) edges and N is the number of nodes

In [ ]:
trees = list()
for H in subgraphs:
    if H.number_of_edges()==H.number_of_nodes()-1 and nx.is_connected(H):
        trees.append(H)

print("Number of spanning trees:", len(trees))

Let us plot some of the spanning trees

In [ ]:
plt.subplot(141)
nx.draw_circular(trees[1], with_labels=True)

plt.subplot(142)
nx.draw_circular(trees[5], with_labels=True)

plt.subplot(143)
nx.draw_circular(trees[9], with_labels=True)

plt.subplot(144)
nx.draw_circular(trees[15], with_labels=True)

# Algebraic graph theory
Algebraic graph theory is concerned with the study of graphs through several related matrices.
We then need to understand how arrays are represented and can be manipulated with Python using the library NumPy.

## Array creation
You can create a numpy array from a Python list or tuple using the  `array ` function

In [ ]:
import numpy as np
a = np.array([2,3,4])
print("a=", a)
print("Dimension of a:", a.shape)

 `array ` transforms lists of lists into two-dimensional arrays, i.e., matrices

In [ ]:
b = np.array([[1.5,2,3], [4,5,6]])
print(b)
print("Dimension of b:", b.shape)

The function  `zeros ` creates an array full of zeros, the function  `ones ` creates an array full of ones

In [ ]:
print(np.zeros((3, 4))) # tuple (3,4) is the shape of the zero array to be created
print(np.ones((2,3)))

One can create arrays whose elements space in a given range using  `arange `

In [ ]:
# the first two arguments are the starting point (included) and the ending point (excluded)
# the third argument is the step-size
np.arange( 10, 30, 5 ) 

or using the function `linspace` that receives as an argument the number of elements that we want to obtain in the given range, instead of the step size

In [ ]:
# the first two arguments are the starting and ending point (both included!), 
# third argument is the number of elements
np.linspace( 0, 2, 9 )


To summarize, numpy `ndarray` can represent arrays of any dimension but we will restrict to dimension 1 (vectors) and 2 (matrices). One-dimensional arrays are printed as rows, bidimensionals as matrices.

In [ ]:
# np.arange(): default start value is 0, default step size is 1
a = np.arange(6)                         # 1d array
print("a:", a, "\n")
b = np.arange(12).reshape(4,3)           # 2d array
print("b: \n",b)

## Basic operations
Arithmetic operators on arrays apply elementwise. A new array is created and filled with the result.

In [ ]:
a = np.array([20,30,40,50])
print("a=",a)
# np.arange(): default start value is 0, default step size is 1, 
# so b = [0,1,2,3]
b = np.arange( 4 )
print("b=", b)
c = a-b
print("c= a-b =", c)
# ** denotes the second power ^2
print("b^2=", b**2) 
print("10 sin(a)=", 10*np.sin(a))
print("a<35:", a<35)

The product operator `*` operates element-wise in NumPy arrays. 
The matrix product can be performed using the `@` operator or the `dot` function:

In [ ]:
# Create two numpy arrays starting from lists of lists
A = np.array( [[1,1],   
               [0,1]] )
B = np.array( [[2,0],
               [3,4]] )
print("A*B= \n",A * B, "\n")                      # elementwise product
print("A@B= \n",A @ B, "\n")                      # matrix product
print("A.dot(B)=\n",A.dot(B))               # another matrix product

NumPy provides familiar mathematical functions such as sin, cos, and exp. In NumPy, these are called “universal functions”(`ufunc`). Within NumPy, these functions operate elementwise on an array, producing an array as output.

In [ ]:
B = np.arange(3) # B=[0,1,2]
np.exp(B)

# Spectral graph theory
We will explore several notions from spectral graph theory by analysing the following graph:

In [ ]:
G = nx.DiGraph()
nx.add_cycle(G,[1,2,3,4])
nx.add_cycle(G,[4,3,2,1])

nx.draw_circular(G, with_labels=True)

First we construct the weight matrix (called adjacency matrix in NetworkX) $W$, the diagonal matrix $D$, and the normalized adjacency matrix $P$

In [ ]:
W = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
# convert W to a numpy array
W = W.toarray()
degrees = np.sum(W,axis=1)
print("Degrees",degrees)
D = np.diag(degrees)
print("D: \n",D)
# P = D^(-1) W
P = np.linalg.inv(D) @ W
print("P: \n",P)

We compute the powers of the matrix $W$. Powers of $W$ have a useful interpretation in a unweighted (di)graphs (the interpretation can be extended to weighted graphs).

**Interpretation**: $(W^n)_{ij}$ equals the number of walks of length n from i to j

In [ ]:
# Compute W^2
#power = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
power = W
# convert W to a numpy array
#power = power.toarray()
power = power @ W
print("W^2 =", power)

In [ ]:
# Compute W^3
power = power @ W
print("W^3 =", power)

# note that we compute P^3 by using P^2 to save computational time

In [ ]:
# Compute W^10
for n in range(3,10):
    power = power @ W
print("W^10 =", power)

The powers of $W$ still contain zero elements, because the graph G is periodic.

Let us modify the graph to make it aperiodic, and compute again the powers of $W$.

In [ ]:
print("Is G aperiodic:",nx.is_aperiodic(G))

In [ ]:
G2 = nx.DiGraph()
G2.add_nodes_from(range(1,3))
nx.add_cycle(G2,[1,2,3,4])
nx.add_cycle(G2,[4,3,2,1])
G2.add_edge(1,3)

print("Is G2 aperiodic:",nx.is_aperiodic(G2))

nx.draw_circular(G2, with_labels=True)

In [ ]:
W2 = nx.adjacency_matrix(G2)
W2 = W2.toarray()
degrees = np.sum(W2,axis=1)
print("Degrees",degrees)
D2 = np.diag(degrees)
print("D2: \n",D2)
P2 = np.linalg.inv(D2) @ W2
print("P2: \n",P2)

**QUESTION**: what is the smallest power of $W$ that does not have null elements?

**HINT**: exploit the fact that in unweighted (di)graphs $(W^n)_{ij}$ is the number of walks between $i$ and $j$.

In [ ]:
N = G2.number_of_nodes()
index = 1

#power = nx.adjacency_matrix(G2) # -> return type is scipy.sparse.csr_matrix
# convert W to a numpy array
#power = power.toarray()
power = W2

while nx.is_aperiodic(G2):
    if np.count_nonzero(power) == N*N:
        print("The smallest power of W with all non-zero elements is:", index)
        break
    else:
        index += 1
        power = power @ W2
        
print(power)

We can also observe that G is bipartite, while G2 is not. To this end, we exploit the notion of coloring.

A coloring is a function $\phi$ that assigns to every node $n$ a color $\phi(n)$ with the properties that for every pair of adjacent nodes $\{n,m\}$, then $\phi(n) != \phi(m)$

### THEOREM: 
A graph is bipartite if and only if there exists a 2-coloring (coloring with 2 colors) defined on the graph.

In [ ]:
# We construct a coloring for the graph G
color_map = []

for node in range(G.number_of_nodes()):
    if node % 2 == 1:
        color_map.append('blue')
    else: 
        color_map.append('red')
        
print(color_map)

nx.draw_circular(G, node_color=color_map, with_labels=True)

Since G admits a coloring, G is bipartite.
Note that a 2-coloring for G2 does not exist, thus G2 is not bipartite.

We validate this observation by using the following result, which is consequence of Perron-Frobenius theorem:

**Theorem**: if a graph is bipartite, then the spectrum of its normalized adjacency matrix P contains -1

In [ ]:
# eig(P.T) returns the eigenvalues (in vector w)
# and eigenvectors (in matrix v) of P'

w,v = np.linalg.eig(P)
print("spectrum of P:",w)

w,v = np.linalg.eig(P2)
print("spectrum of P2:",w)

The trace of the n-th power of $W$ equals the number of circuits with length n in the graph.

**Question**: what is the trace of $W^3$?

To answer the question, let us plot again G

In [ ]:
nx.draw_circular(G, with_labels=True)

In [ ]:
print("Number of length-3 circuits in G:", np.matrix.trace(W@W@W))
print("Number of length-3 circuits in G2:", np.matrix.trace(W2@W2@W2))

We now define a new graph.

In [ ]:
G = nx.DiGraph()
G.add_nodes_from(range(1,11))
nx.add_cycle(G,[1,2,3])
nx.add_cycle(G,[4,5,6])
nx.add_cycle(G,[8,9,10])
G.add_edges_from([(4,3), (3,8), (5,7), (7,7)])

# define pos according to spring layout
# to fix nodes' positions in all graph drawings.
# spring_layout positions nodes using Fruchterman-Reingold force-directed algorithm.
pos = nx.spring_layout(G)
nx.draw(G,pos, with_labels=True)

## Invariant probability distributions
In this section we show how to compute the invariant distributions of a graph

In [ ]:
W = nx.adjacency_matrix(G)
W = W.toarray()
degrees = np.sum(W,axis=1)
print("Degrees",degrees)
D = np.diag(degrees)
print("D: \n",D)
# P = D^(-1) W
P = np.linalg.inv(D) @ W
print("P: \n",P)

Then we compute the invariant distributions of G, i.e., the normalized eigenvectors of P' relative to eigenvalue 1, which solve
$$
P' \pi = \pi, \quad \sum_i \pi_i = 1
$$ To do this, first we compute all eigenvalues and eigenvectors of P' with function `np.linalg.eig()`:

**THEOREM**: if the graph G is strongly connected, the eigenvalue 1 has multeplicity 1

We compute the condensation graph of G

In [ ]:
CG = nx.algorithms.components.condensation(G)
nx.draw(CG, with_labels=True)

print(dict(CG.nodes))

The graph is not strongly connected, since the condensation graph has more than one node. 

To confirm this, we compute the spectrum of $P$, and show that the eigenvalue 1 has algebraic multeplicity greater than 1.

In [ ]:
# eig(P.T) returns the eigenvalues (in vector w)
# and eigenvectors (in matrix v) of P'
w,v = np.linalg.eig(P.T)
print("eigenvalues:",w) # -> the 0th and 5th eigenvalues are 1

There are two eigenvalues equal to 1 (in position 0 and 5 of `w`). We select the eigenvectors corresponding to the two occurrencies of eigenvalue 1 and we normalize them to obtain the two invariant distributions (`pi0` and `pi5`):

In [ ]:
# we iterate over indices corresponding to eigenvalues 1
# i.e. corresponding to entries of w that are equal 1:
# for each index we extract the corresponding eigenvector in v
# and normalize it

# we use np.isclose() to compare eigenvalues to 1 to avoid
# numerical precision errors
for index in [i for i in range(len(G)) if np.isclose(w[i],1)]: 
    pi = v[:,index].real  # -> eigenvectors are complex but pi is real, so we convert it to real
    pi = pi/np.sum(pi)
    print("pi", index, "=", pi)

**Theorem**: the multeplicity of the eigenvalue 1 equals the number of attractive components (or trapping sets, as defined in the lecture notes) of the graph. Moreover, for every attractive component, there exists a dominant eigenvector (called extremal) whose support is exactly the node set of the attractive component.

#### Exercise

Perform the computation of the extremal eigenvectors of G.

1. Find the attractive components of G
2. For each attractive component, construct the corresponding induced subgraph
3. Compute the P matrix of each induced subgraph and its invariant measure
4. Map the obtained measures back to the original graph G (by adding zeros in the appropriate positions)

**Hint**: use the methods introduced in the previous notebook, and the code above.

In [ ]:
# TO DO

Up to now we have exploited the function `numpy.linalg.eig` to compute all the eigenvalues and eigenvectors of P', and we have selected the leading ones. 

Another approach consists in applying an iterative method which converges to the leading eigenvector. This will be illustrated in the following weeks.

# Network centralities

Node centralities measure the importance of the nodes of the network. Several notions of centrality may be defined, e.g.:

**Degree centrality**: the centrality of a node is proportional to its degree. In digraphs, we can define both the indegree and the outdegree centrality. For instance, the indegree centrality is a measure of importance in Twitter network, since it measures the number of followers of every account.

**Eigenvector centrality**: the eigenvector centrality generalizes the degree centrality. Instead of counting the number of neighbors as the degree centrality does, this centrality gives more importance to connections with more central nodes. Let $z$ denote the centrality. Thus,

$$
z_i \propto \sum_{j} W_{ji}z_j
$$

By normalizing $z$, and taking the proportionality factor equal to dominant eigenvalue $\lambda_W$, we obtain $z = W'z$, i.e., $z$ is the dominant eigenvector of $W$, and has non-negative components because of Perron-Frobenius theorem.



## Small networks Example: Zachary's Karate Club
Zachary's Karate Club network is a well-know network example. This is a quite small network so we can compute centralities direcly. To better understand the meaning of centrality measure it is useful to visualize them by producing appropriate graph representations.

Let's first load and visualize Zachary's Karate Club network:

In [ ]:
G = nx.karate_club_graph()

# Fix node positions on all pictures according to spring layout
pos = nx.spring_layout(G) 
nx.draw_networkx(G, pos)

# Visualizing centralities on graphs
We can compute centralities both using NetworkX algorithms and performing iterative procedures. It is important to make sense of the centrality vectors, and a useful way to do this is by visualizing centralities on graphs.

Compute, for example, the degree centrality of G. The following code shows how to represent G so that nodes size and color reflects their centrality value:

In [ ]:
import matplotlib.pyplot as plt

# Degree centrality

# dc is a dictionary with nodes as keys and degree centralities as values
dc = nx.degree_centrality(G) 
plt.figure(1, figsize=(10,7))
# we draw the graph with same node position "pos" defined above
nx.draw(G,pos,
         with_labels=True,
         # keys of dc are nodes
         nodelist=dc.keys(), 
         # node size is proportional to centrality value
         node_size = [d*7000 for d in dc.values()], 
         # node's color reflects centrality values (higher dc = darker color)
         node_color=list(dc.values()),
         font_size=8,
         # node's colors are on the red scale
         cmap=plt.cm.Reds) 

In [ ]:
plt.figure(1, figsize=(10,7))
# we draw the graph with same node position "pos" defined above
nx.draw(G,pos,
         with_labels=True,
         # keys of dc are nodes
         nodelist=dc.keys(), 
         # node size is proportional to centrality value
         node_size = [d*7000 for d in dc.values()], 
         # node's color reflects centrality values (higher dc = darker color)
         node_color=list(dc.values()),
         font_size=8,
         # node's colors are on the red scale
         cmap=plt.cm.Reds) 

We repeat this procedure with a different measure, eigenvector centrality:

In [ ]:
# Eigenvector centrality
ec = nx.eigenvector_centrality(G)
plt.figure(1, figsize=(10,7))
nx.draw(G, pos,
          with_labels=True,
          nodelist=ec.keys(),
          # node size is proportional to eigenvector centrality
          node_size = [d*7000 for d in ec.values()],  
          node_color=list(ec.values()),
          font_size=8,
          cmap=plt.cm.Reds,
          )

It is interesting to compare different centrality measures for the same graph and see how they are correlated. Below we visualize the correlation between degree centrality and eigenvector centrality of G:

In [ ]:
# Correlation degree-eigenvector

# x corresponds to degree centrality values
xdata = list(dc.values()) 
# y corresponds to eigenvector centrality values
ydata = list(ec.values()) 

plt.figure(1, figsize=(7,7))
# for each node, we plot an "+" with coordinates equal to the values of its
# degree and eigenvector centrality
plt.plot(xdata,ydata, '+') 
plt.xlabel('Degree Centrality')
plt.ylabel('Eigenvector Centrality')

The two centralities appear to be correlated for G. To explore this in more details it is usefull to add node ids, so that we can see which are the nodes with higher or lower correlation. Before doing that, answer the following question.

**Question**: where do you expect to see nodes 11 and 16 in the degree-eigenvector correlation above? Can you identify them?

In [ ]:
# Adding node ids:

# We define a figure and we construct two subplots: 
# on the left we plot the centralities correlation diagram
# with node labels, on the right we draw the graph 
# with same node labels
fig = plt.figure(1, figsize=(14,7))
# add_subplot() returns the axes of the subplot
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

for v in range(len(G)):
    # Axes.text(x,y,s) add the text s to Axes instance (i.e., to the subplot)
    # at location x, y. For each node v we plot 
    # node ids in position (xdata[v], ydata[v]) where xdata = list(dc.values())
    # and ydata = list(ec.values())
    ax1.text(x = xdata[v], y = ydata[v], s=str(v))
# we set the limits for x and y scales

ax1.set_xlim(0, 0.6)
ax1.set_ylim(0, 0.4)
ax1.set_xlabel('Degree Centrality')
ax1.set_ylabel('Eigenvector Centrality')

nx.draw_networkx(G, pos, ax=ax2)

Other centralities include:
- the invariant distribution centrality, which weights more connections with nodes that has less connections, i.e.
$$
z_i=\sum_{j} P_{ji} z_j
$$
- the Bonacich centrality, which generalizes the invariant distribution centrality by assigning an intrinsic centrality to every nodes and considering both intrinsic centrality and network effects. A particular version of this centrality measure is known as Page rank centrality measure, and is used as a centrality of measures of webpages in the WWW.

In the next lecture we will analyze also these centralities, and propose iterative algorithms to compute them.

In [ ]:
attr_components = tuple(nx.algorithms.components.attracting_components(G))

for c in attr_components:
    # construct the induced subgraph with nodes from the attractive component c
    sG = G.subgraph(c)
    # construct the matrix P on the subgraph
    W = nx.adjacency_matrix(sG)
    W = W.toarray()
    degrees = np.sum(W,axis=1)
    D = np.diag(degrees)
    P = np.linalg.inv(D) @ W
    # find the extremal dominant eigenvector corresponding to component c
    w,v = np.linalg.eig(P.T)
    for index in [i for i in range(len(sG)) if np.isclose(w[i],1)]: 
        pi = v[:,index].real  # -> eigenvectors are complex but pi is real, so we convert it to real
        pi = pi/np.sum(pi)
    # map pi back in the original node space
    pi_G = np.zeros(len(G))
    for i in range(len(sG)):
        pi_G[list(sG.nodes)[i]-1] = pi[i] # shift by -1 because nodes are (1,...10) while vector indexes are (0,...,9)
    print("pi:", pi_G)
    